In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

In [39]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if not self.is_test:
            # Fetch training images using paths from train.csv
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            label = int(self.data.iloc[idx, 0].split('_')[1]) - 1  # Labeling
            label = torch.tensor(label)
            if self.transform:
                image = self.transform(image)
            return image, label
        else:
            # For test data
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.data.iloc[idx, 0]

# Densenet121

In [40]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/train.csv',  # Correct path to train.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/train',  # Directory containing images
    transform=transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/test.csv',  # Correct path to test.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/test/',  # Directory containing test images
    transform=transform, is_test=True
)

In [41]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [42]:
class CustomDenseNet(nn.Module):
    def __init__(self, num_classes=71):  # Ensure correct number of classes
        super(CustomDenseNet, self).__init__()

        # Load pre-trained DenseNet121 model
        self.base_model = models.densenet121(pretrained=True)

        # Freeze all layers if needed (Optional)
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Replace the classifier part of the DenseNet
        in_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),  # Hidden layer
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        x = self.base_model(x)
        return x

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet(num_classes=71).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [47]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.3287, Accuracy: 0.8914
Validation Accuracy: 0.8950
Training - Epoch 2/15, Loss: 0.3086, Accuracy: 0.9061
Validation Accuracy: 0.8937
Training - Epoch 3/15, Loss: 0.2952, Accuracy: 0.9043
Validation Accuracy: 0.8912
Training - Epoch 4/15, Loss: 0.2832, Accuracy: 0.9050
Validation Accuracy: 0.8893
Training - Epoch 5/15, Loss: 0.2644, Accuracy: 0.9147
Validation Accuracy: 0.9038
Training - Epoch 6/15, Loss: 0.2545, Accuracy: 0.9201
Validation Accuracy: 0.8874
Training - Epoch 7/15, Loss: 0.2524, Accuracy: 0.9171
Validation Accuracy: 0.8887
Training - Epoch 8/15, Loss: 0.2357, Accuracy: 0.9261
Validation Accuracy: 0.9063
Training - Epoch 9/15, Loss: 0.2206, Accuracy: 0.9294
Validation Accuracy: 0.8969
Training - Epoch 10/15, Loss: 0.2008, Accuracy: 0.9334
Validation Accuracy: 0.8950
Training - Epoch 11/15, Loss: 0.1950, Accuracy: 0.9371
Validation Accuracy: 0.8937
Training - Epoch 12/15, Loss: 0.2074, Accuracy: 0.9284
Validation Accuracy: 0.8950
Training - Ep

In [48]:
for param in model.base_model.features[-1:].parameters():  
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.1539, Accuracy: 0.9515
Validation Accuracy: 0.8943
Training - Epoch 2/15, Loss: 0.1494, Accuracy: 0.9511
Validation Accuracy: 0.8962
Training - Epoch 3/15, Loss: 0.1428, Accuracy: 0.9547
Validation Accuracy: 0.8975
Training - Epoch 4/15, Loss: 0.1426, Accuracy: 0.9547
Validation Accuracy: 0.8931
Training - Epoch 5/15, Loss: 0.1448, Accuracy: 0.9512
Validation Accuracy: 0.8987
Training - Epoch 6/15, Loss: 0.1382, Accuracy: 0.9531
Validation Accuracy: 0.9019
Training - Epoch 7/15, Loss: 0.1356, Accuracy: 0.9574
Validation Accuracy: 0.9006
Training - Epoch 8/15, Loss: 0.1408, Accuracy: 0.9547
Validation Accuracy: 0.8994
Training - Epoch 9/15, Loss: 0.1396, Accuracy: 0.9556
Validation Accuracy: 0.8962
Training - Epoch 10/15, Loss: 0.1287, Accuracy: 0.9630
Validation Accuracy: 0.8994
Training - Epoch 11/15, Loss: 0.1355, Accuracy: 0.9585
Validation Accuracy: 0.8969
Training - Epoch 12/15, Loss: 0.1281, Accuracy: 0.9607
Validation Accuracy: 0.8943
Training - Ep

In [49]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [50]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission8.csv', index=False)

# Resnet50

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/train.csv',  # Correct path to train.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/train',  # Directory containing images
    transform=transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/test.csv',  # Correct path to test.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/test/',  # Directory containing test images
    transform=transform, is_test=True
)

In [4]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomResNet50, self).__init__()

        # Load pre-trained ResNet50 model
        self.base_model = models.resnet50(pretrained=True)

        # Freeze the feature extraction layers
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Modify the classifier (fully connected layer)
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 256),  # Fully connected layer from in_features to 256
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        x = self.base_model(x)
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomResNet50(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 167MB/s] 


In [7]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [8]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 3.0821, Accuracy: 0.2637
Validation Accuracy: 0.7597
Training - Epoch 2/10, Loss: 1.5688, Accuracy: 0.5459
Validation Accuracy: 0.8591
Training - Epoch 3/10, Loss: 1.2159, Accuracy: 0.6353
Validation Accuracy: 0.8516
Training - Epoch 4/10, Loss: 1.0459, Accuracy: 0.6757
Validation Accuracy: 0.8818
Training - Epoch 5/10, Loss: 0.9307, Accuracy: 0.7048
Validation Accuracy: 0.8862
Training - Epoch 6/10, Loss: 0.8723, Accuracy: 0.7267
Validation Accuracy: 0.8610
Training - Epoch 7/10, Loss: 0.8301, Accuracy: 0.7354
Validation Accuracy: 0.8811
Training - Epoch 8/10, Loss: 0.7700, Accuracy: 0.7602
Validation Accuracy: 0.8893
Training - Epoch 9/10, Loss: 0.7446, Accuracy: 0.7667
Validation Accuracy: 0.8918
Training - Epoch 10/10, Loss: 0.7298, Accuracy: 0.7613
Validation Accuracy: 0.8975


In [10]:
for param in model.base_model.layer4.parameters():  # layer4 is the last block in ResNet50
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 0.6653, Accuracy: 0.7851
Validation Accuracy: 0.9063
Training - Epoch 2/10, Loss: 0.6287, Accuracy: 0.8046
Validation Accuracy: 0.9075
Training - Epoch 3/10, Loss: 0.5602, Accuracy: 0.8243
Validation Accuracy: 0.9082
Training - Epoch 4/10, Loss: 0.5553, Accuracy: 0.8217
Validation Accuracy: 0.9126
Training - Epoch 5/10, Loss: 0.5290, Accuracy: 0.8254
Validation Accuracy: 0.9145
Training - Epoch 6/10, Loss: 0.4985, Accuracy: 0.8390
Validation Accuracy: 0.9151
Training - Epoch 7/10, Loss: 0.4821, Accuracy: 0.8438
Validation Accuracy: 0.9151
Training - Epoch 8/10, Loss: 0.4665, Accuracy: 0.8466
Validation Accuracy: 0.9195
Training - Epoch 9/10, Loss: 0.4562, Accuracy: 0.8516
Validation Accuracy: 0.9214
Training - Epoch 10/10, Loss: 0.4414, Accuracy: 0.8499
Validation Accuracy: 0.9220


In [11]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            # Chuyển predicted_classes từ số thành tên class
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
            # Lưu lại image_ids từ test_loader thay vì sử dụng tensor index
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

# Map số index về class_name
class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [12]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)

In [13]:
renamed_ids = list(range(len(ids)))

In [14]:
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission2.csv', index=False)

In [15]:
results

,ID,TARGET
0,0,class_58
1,1,class_58
2,2,class_7
3,3,class_17
4,4,class_71
...,...,...
1395,1395,class_69
1396,1396,class_51
1397,1397,class_65
1398,1398,class_49


# Resnet 50_Version 2

In [51]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.CenterCrop(224),     
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [52]:

# Create datasets
train_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/train.csv',  # Correct path to train.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/train',  # Directory containing images
    transform=transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/test.csv',  # Correct path to test.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/test/',  # Directory containing test images
    transform=transform, is_test=True
)

In [53]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [54]:
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomResNet50, self).__init__()

        # Load pre-trained ResNet50 model
        self.base_model = models.resnet50(pretrained=True)

        # Freeze the feature extraction layers
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Modify the classifier (fully connected layer)
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 256),  # Fully connected layer from in_features to 256
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        x = self.base_model(x)
        return x

In [55]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomResNet50(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [56]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [58]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 1.1945, Accuracy: 0.6361
Validation Accuracy: 0.8623
Training - Epoch 2/15, Loss: 1.1307, Accuracy: 0.6487
Validation Accuracy: 0.8296
Training - Epoch 3/15, Loss: 1.0578, Accuracy: 0.6704
Validation Accuracy: 0.8692
Training - Epoch 4/15, Loss: 1.0025, Accuracy: 0.6858
Validation Accuracy: 0.8736
Training - Epoch 5/15, Loss: 0.9555, Accuracy: 0.6956
Validation Accuracy: 0.8654
Training - Epoch 6/15, Loss: 0.9250, Accuracy: 0.7097
Validation Accuracy: 0.8748
Training - Epoch 7/15, Loss: 0.8955, Accuracy: 0.7132
Validation Accuracy: 0.8597
Training - Epoch 8/15, Loss: 0.8757, Accuracy: 0.7173
Validation Accuracy: 0.8616
Training - Epoch 9/15, Loss: 0.8711, Accuracy: 0.7201
Validation Accuracy: 0.8780
Training - Epoch 10/15, Loss: 0.8268, Accuracy: 0.7340
Validation Accuracy: 0.8767
Training - Epoch 11/15, Loss: 0.8182, Accuracy: 0.7360
Validation Accuracy: 0.8805
Training - Epoch 12/15, Loss: 0.8275, Accuracy: 0.7399
Validation Accuracy: 0.8780
Training - Ep

In [59]:
for param in model.base_model.layer4.parameters():  # layer4 is the last block in ResNet50
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.7439, Accuracy: 0.7588
Validation Accuracy: 0.8937
Training - Epoch 2/15, Loss: 0.6815, Accuracy: 0.7816
Validation Accuracy: 0.8899
Training - Epoch 3/15, Loss: 0.6258, Accuracy: 0.7936
Validation Accuracy: 0.8918
Training - Epoch 4/15, Loss: 0.5995, Accuracy: 0.8038
Validation Accuracy: 0.9025
Training - Epoch 5/15, Loss: 0.5535, Accuracy: 0.8143
Validation Accuracy: 0.8906
Training - Epoch 6/15, Loss: 0.5557, Accuracy: 0.8158
Validation Accuracy: 0.8975
Training - Epoch 7/15, Loss: 0.5236, Accuracy: 0.8254
Validation Accuracy: 0.8981
Training - Epoch 8/15, Loss: 0.5299, Accuracy: 0.8254
Validation Accuracy: 0.8950
Training - Epoch 9/15, Loss: 0.4961, Accuracy: 0.8302
Validation Accuracy: 0.9044
Training - Epoch 10/15, Loss: 0.4906, Accuracy: 0.8342
Validation Accuracy: 0.9044
Training - Epoch 11/15, Loss: 0.4788, Accuracy: 0.8417
Validation Accuracy: 0.9050
Training - Epoch 12/15, Loss: 0.4696, Accuracy: 0.8409
Validation Accuracy: 0.9063
Training - Ep

In [60]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            # Chuyển predicted_classes từ số thành tên class
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
            # Lưu lại image_ids từ test_loader thay vì sử dụng tensor index
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

# Map số index về class_name
class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [61]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)

In [62]:
renamed_ids = list(range(len(ids)))

In [63]:
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission5.csv', index=False)

# EfficientNet

In [71]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/train.csv',  # Correct path to train.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/train',  # Directory containing images
    transform=transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/test.csv',  # Correct path to test.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/test/',  # Directory containing test images
    transform=transform, is_test=True)


In [72]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [73]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomEfficientNet, self).__init__()

        # Load EfficientNet-B0 pre-trained model
        self.base_model = models.efficientnet_b1(pretrained=True)
        
        # Modify the final fully connected layer
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

In [74]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomEfficientNet(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 49.0MB/s]


In [75]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [76]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20)

Training - Epoch 1/20, Loss: 2.7936, Accuracy: 0.2851
Validation Accuracy: 0.5748
Training - Epoch 2/20, Loss: 1.2039, Accuracy: 0.6403
Validation Accuracy: 0.7063
Training - Epoch 3/20, Loss: 0.7447, Accuracy: 0.7697
Validation Accuracy: 0.7396
Training - Epoch 4/20, Loss: 0.5532, Accuracy: 0.8307
Validation Accuracy: 0.7893
Training - Epoch 5/20, Loss: 0.4303, Accuracy: 0.8644
Validation Accuracy: 0.7748
Training - Epoch 6/20, Loss: 0.3600, Accuracy: 0.8897
Validation Accuracy: 0.7912
Training - Epoch 7/20, Loss: 0.3021, Accuracy: 0.9056
Validation Accuracy: 0.7616
Training - Epoch 8/20, Loss: 0.2832, Accuracy: 0.9133
Validation Accuracy: 0.8151
Training - Epoch 9/20, Loss: 0.2447, Accuracy: 0.9257
Validation Accuracy: 0.7887
Training - Epoch 10/20, Loss: 0.2480, Accuracy: 0.9246
Validation Accuracy: 0.8107
Training - Epoch 11/20, Loss: 0.2351, Accuracy: 0.9308
Validation Accuracy: 0.7881
Training - Epoch 12/20, Loss: 0.2106, Accuracy: 0.9402
Validation Accuracy: 0.7673
Training - Ep

In [77]:
for param in model.base_model.features[6:].parameters():
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20)

Training - Epoch 1/20, Loss: 0.0985, Accuracy: 0.9722
Validation Accuracy: 0.8277
Training - Epoch 2/20, Loss: 0.0851, Accuracy: 0.9747
Validation Accuracy: 0.8308
Training - Epoch 3/20, Loss: 0.0923, Accuracy: 0.9731
Validation Accuracy: 0.8314
Training - Epoch 4/20, Loss: 0.0788, Accuracy: 0.9767
Validation Accuracy: 0.8314
Training - Epoch 5/20, Loss: 0.0839, Accuracy: 0.9755
Validation Accuracy: 0.8340
Training - Epoch 6/20, Loss: 0.0805, Accuracy: 0.9769
Validation Accuracy: 0.8377
Training - Epoch 7/20, Loss: 0.0715, Accuracy: 0.9789
Validation Accuracy: 0.8377
Training - Epoch 8/20, Loss: 0.0677, Accuracy: 0.9770
Validation Accuracy: 0.8377
Training - Epoch 9/20, Loss: 0.0712, Accuracy: 0.9784
Validation Accuracy: 0.8308
Training - Epoch 10/20, Loss: 0.0588, Accuracy: 0.9817
Validation Accuracy: 0.8403
Training - Epoch 11/20, Loss: 0.0640, Accuracy: 0.9814
Validation Accuracy: 0.8428
Training - Epoch 12/20, Loss: 0.0652, Accuracy: 0.9800
Validation Accuracy: 0.8440
Training - Ep

In [78]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission6.csv', index=False)

In [79]:
results

,ID,TARGET
0,0,class_58
1,1,class_58
2,2,class_7
3,3,class_17
4,4,class_71
...,...,...
1395,1395,class_69
1396,1396,class_51
1397,1397,class_56
1398,1398,class_30
